# Progetto di Foundations of Computer Science - 2019

####  Importo le librerie necessarie

In [1]:
import pandas as pd
import numpy
import re

##### Leggo i file CSV dal PC di Matteo

In [4]:
ll = pd.read_csv(r"/home/mcampironi/Documenti/Data Science/Foundations of CS/Dataset/loans_lenders.csv")

In [2]:
loans = pd.read_csv(r"/home/mcampironi/Documenti/Data Science/Foundations of CS/Dataset/loans.csv")

In [3]:
GEconV4 = pd.read_csv(r"/home/mcampironi/Documenti/Data Science/Foundations of CS/Dataset/GEconV4.csv", sep=';')

In [4]:
loan_coords = pd.read_csv(r"/home/mcampironi/Documenti/Data Science/Foundations of CS/Dataset/loan_coords.csv")

####  1. Normalize the *loan_lenders* table. In the normalized table, each row must have one *loan_id* and one *lender*.

In [5]:
df = (ll.set_index(['loan_id']).apply(lambda x: x.str.split(',').explode()).reset_index())

####  2. For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing

In [98]:
loans['duration'] = (pd.to_datetime(loans.planned_expiration_time)-pd.to_datetime(loans.disburse_time)).dt.days

#### 3. Find the lenders that have funded at least twice.

In [11]:
fund = df.groupby(['lenders']).size().reset_index()
fund.columns = ['lenders', 'num_of_fund']

In [23]:
fund[fund['num_of_fund'] >= 2]

,lenders,num_of_fund
0,000,39
1,00000,39
2,0002,70
4,0101craign0101,71
5,0132575,4
...,...,...
1639018,zyrorl,3
1639020,zzaman,11
1639021,zzanita,2
1639024,zzmcfate,56


#### 4. For each country, compute how many loans have involved that country as borrowers.

Rinomino le colonne di country_coords. Inoltre arrotondo le coordinate in loan_coords e le rendo intere in modo da poter effettuare il merge.

In [5]:
country_coords = GEconV4[['COUNTRY','LAT','LONGITUDE']]
country_coords.columns = ['country','latitude','longitude']

In [6]:
loan_coords[['latitude','longitude']]=loan_coords[['latitude','longitude']].apply(round).astype('int64')

Effettuo il merge in modo da ottenere un dataframe che contenga i loan_id per i rispettivi paesi

In [7]:
loan_country = pd.merge(loan_coords, country_coords.drop_duplicates(subset=['latitude','longitude']), how='left', on=['latitude','longitude'])
loan_country = loan_country.drop(columns = ['latitude','longitude'])

Controllo che lo status sia "funded"

In [12]:
loans_funded = loans[loans['status'] == 'funded']

Creo un dataframe contenente per ogni paese il totale di soldi finanziati

In [121]:
country_loans = pd.merge(loan_country, loans_funded[['loan_id','funded_amount']], how='left', on='loan_id')
country_loans = country_loans.groupby('country').size().reset_index()
country_loans.columns = ['country','total_loan']
country_loans

,country,total_loan
0,Afghanistan,6681
1,Albania,1079
2,Angola,5
3,Argentina,4578
4,Armenia,12927
...,...,...
105,Uruguay,1
106,Vietnam,15828
107,Yemen,3304
108,Zambia,712
